<h4>Web-scraped Toronto Neighbourhood Data</h4>

<h5>Import Dependencies</h5>

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
import csv

from bs4 import BeautifulSoup as bs # Beautiful Soup v4

# May be required for direct read of table tp Pandas
!pip install lxml
!pip install html5lib

# Only use when full dataframe needs to be viewed
#pd.set_option("display.max_rows", None, "display.max_columns", None)

!pip install -c conda-forge geopy --yes 

!pip install geocoder
import geocoder # import geocoder

#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Folium installed
Libraries imported.


<h5>Extract contents of web table</h5>

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).text

table = pd.read_html(html)[0]

<h5>Create and instantiate the Dataframe</h5>

In [3]:
# Insert table data directly into Dataframe

t_neighbourhoods = pd.DataFrame(data=table)

# Rename column

t_neighbourhoods = t_neighbourhoods.rename(columns={'Postal Code': 'PostalCode'})

# View Dataframe shape and head

print(t_neighbourhoods.shape)
t_neighbourhoods.head()

(180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h5>Clean the Dataframe by deleting rows with unassigned data</h5>

In [4]:
# Generate row indices of target items

ItemIndices = t_neighbourhoods[t_neighbourhoods.Borough == 'Not assigned'].index

# Delete these row indices from dataFrame

t_neighbourhoods.drop(ItemIndices, inplace = True)

# View Dataframe shape and head

print(t_neighbourhoods.shape)
t_neighbourhoods.head()

(103, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>Reset the Dataframe index</h5>

In [5]:
t_neighbourhoods.reset_index(drop = True, inplace=True)

# View Dataframe head

t_neighbourhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>View Dataframe shape</h5>

In [6]:
print(t_neighbourhoods.shape)

(103, 3)


<h5>Create Latitude and Longitude Dataframe</h5>

In [7]:
LatLon = pd.DataFrame (columns = ['Latitude','Longitude'])
LatLon

,Latitude,Longitude


<h5>Obtain Latitude and Longitude values from Geocoder and place in LatLon</h5>

In [8]:
# Geocoder does not work so this cell contents have all been commented out

#for row in t_neighbourhoods.itertuples(index = False, name = 'PostalCode'):
#    code = row.PostalCode

    # initialize your variable to None
#    lat_lng_coords = None

    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format('M3A'))
#        lat_lng_coords = g.latlng

#    LatLon.Latitude = lat_lng_coords[0]
#    LatLon.Longitude = lat_lng_coords[1]
    
#LatLon    

<h5>Latitude and Longitude datafile downloaded from web repository</h5>

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

<h5>Latitude and Longitude values are placed in DataFrame</h5>

In [10]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h5>The two DataFrames are merged</h5>

In [11]:
T_neighbourhoods = t_neighbourhoods.merge(coordinates, left_on = 'PostalCode', right_on = 'Postal Code')[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
T_neighbourhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
